# ROLLING REGRESSION OF *LOGARITHMIC RETURNS*

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.regression.rolling import RollingOLS
import statsmodels.api as sm
from pandas_datareader import data as pdr

C:\Users\Alfred\AppData\Local\Temp\ipykernel_8784\194951791.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


We start by importing our data acquired in the previous notebooks and parsing dates:


In [ ]:
nas100 = pd.read_csv("Data/nas100.csv")
nas100.Date = pd.to_datetime(nas100.Date)
nas100.set_index("Date", inplace = True)

log_returns = pd.read_csv("Data/StocksReturns.csv")
log_returns.Date = pd.to_datetime(log_returns.Date)
log_returns.set_index("Date", inplace = True)

log_returns.head()

,ADBE,ADP,ABNB,GOOGL,GOOG,AMZN,AMD,AEP,AMGN,ADI,...,SRCL,WOR,STRM,ASAI,CHRS,CA,AAL,LOGI,CHKP,LBTYA
Date,,,,,,,,,,,,,,,,,,,,,
2013-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-02,-0.009902,-0.011702,NaN,-0.006796,NaN,-0.002058,0.020461,-0.013137,0.014965,-0.032934,...,-0.011167,-0.010992,-0.033506,NaN,NaN,NaN,0.004347,-0.025149,-0.001862,-0.002926
2014-01-03,-0.002195,0.010835,NaN,-0.007322,NaN,-0.003852,0.012579,-0.000434,-0.011552,0.006674,...,0.001479,0.025852,0.011782,NaN,NaN,NaN,0.045480,0.003738,-0.001399,-0.001579
2014-01-06,-0.017736,-0.011461,NaN,0.011088,NaN,-0.007113,0.031983,-0.000651,-0.008686,-0.005660,...,-0.002611,-0.018193,-0.005874,NaN,NaN,NaN,0.018294,0.014080,-0.005772,0.015456
2014-01-07,0.014519,0.012081,NaN,0.019095,NaN,0.011116,0.012034,0.006489,0.025664,0.005257,...,0.011351,0.002857,0.008798,NaN,NaN,NaN,-0.004449,0.033288,0.000313,-0.002337


In [ ]:
dates = nas100.index
dates

DatetimeIndex(['2013-12-31', '2014-01-02', '2014-01-03', '2014-01-06',
               '2014-01-07', '2014-01-08', '2014-01-09', '2014-01-10',
               '2014-01-13', '2014-01-14',
               ...
               '2023-12-15', '2023-12-18', '2023-12-19', '2023-12-20',
               '2023-12-21', '2023-12-22', '2023-12-26', '2023-12-27',
               '2023-12-28', '2023-12-29'],
              dtype='datetime64[ns]', name='Date', length=2517, freq=None)

We download the values of Fama and French Three Factor Model to extract the risk-free rate, useful for the following steps:

In [ ]:
ff_factors = pdr.get_data_famafrench('F-F_Research_Data_Factors_daily', start = "2014-01-01", end = "2023-12-31")[0]
ff_factors = ff_factors / 100  # Convert to decimal
ff_factors

C:\Users\Alfred\AppData\Local\Temp\ipykernel_8784\3145656080.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_factors = pdr.get_data_famafrench('F-F_Research_Data_Factors_daily', start = "2014-01-01", end = "2023-12-31")[0]


,Mkt-RF,SMB,HML,RF
Date,,,,
2014-01-02,-0.0088,-0.0028,0.0018,0.00000
2014-01-03,0.0003,0.0036,0.0004,0.00000
2014-01-06,-0.0034,-0.0057,0.0029,0.00000
2014-01-07,0.0068,0.0038,-0.0037,0.00000
2014-01-08,0.0004,-0.0001,-0.0011,0.00000
...,...,...,...,...
2023-12-22,0.0020,0.0064,0.0010,0.00021
2023-12-26,0.0048,0.0071,0.0043,0.00021
2023-12-27,0.0016,0.0014,0.0010,0.00021


We calculated stocks and index returns net of the risk-free rate, as prescribed by the single index model used:

In [ ]:
risk_free_rate = ff_factors['RF']
excess_log_returns = log_returns.subtract(risk_free_rate, axis = 0)
excess_market_log_returns  = nas100["log_returns"].subtract(risk_free_rate, axis = 0)

excess_market_log_returns.head()

Date
2013-12-31         NaN
2014-01-02   -0.007946
2014-01-03   -0.006995
2014-01-06   -0.003332
2014-01-07    0.008720
dtype: float64

In [ ]:
tickers = log_returns.columns.tolist()
results = dict()
rolling_window = 180

Now we can execute a rolling regression of log-returns for all the stocks:

In [ ]:
for ticker in tickers:
    df_returns = excess_log_returns[ticker].dropna()
    if not df_returns.empty:
        starting_index = df_returns.index[0]
        market_returns = excess_market_log_returns.loc[starting_index:].dropna()
        if not market_returns.empty:
            # Aligning market data indices with ticker returns data indices
            aligned_df = df_returns.align(market_returns, join = 'inner')
            if aligned_df[0].shape[0] > rolling_window:  # this ensures that there is enough data for the rolling window;
                try:
                    model = RollingOLS(aligned_df[0], sm.add_constant(aligned_df[1]), window = rolling_window)
                    result = model.fit()
                    results[ticker] = result
                except Exception as e:
                    print(f"Error for the ticker {ticker}: {e}")
            else:
                print(f"Not enough data for the rolling window for the ticker {ticker}")
                tickers.remove(ticker)
        else:
            print(f"Market data are blank for the ticker {ticker}")
    else:
        print(f"Returns data are empty for the ticker {ticker}")

Not enough data for the rolling window for the ticker ARM


c:\Users\Alfred\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\regression\rolling.py:272: RuntimeWarning: divide by zero encountered in log
  llf = -np.log(ssr) * nobs2  # concentrated likelihood


Not enough data for the rolling window for the ticker FISV
Not enough data for the rolling window for the ticker CA


We can now extract all the parameters of the rolling regression we are interested in:

In [ ]:
# Step 5: Creating a dictionary to store the parameters
params = {
    'R_squared': {},
    'variance_stock': {},
    'mean_return_real': {},
    'alpha': {},
    'beta': {},
    'beta_squared_sigma_m_squared': {},
    'variance_residual': {},
}

In [ ]:
# extracting parameters for each stock
for ticker in tickers:
    # Check if there are results for the ticker
    if ticker in results:
        # Rolling R_squared
        params['R_squared'][ticker] = results[ticker].rsquared

        # Uses logarithmic excess returns
        params['variance_stock'][ticker] = excess_log_returns[ticker].rolling(rolling_window).var()
        params['mean_return_real'][ticker] = excess_log_returns[ticker].rolling(rolling_window).mean()

        params['alpha'][ticker] = results[ticker].params["const"]
        params['beta'][ticker] = results[ticker].params[0]

        params['beta_squared_sigma_m_squared'][ticker] = results[ticker].ess / rolling_window
        params['variance_residual'][ticker] = results[ticker].ssr / rolling_window

# Concatenate the DataFrames, filling with NaN where necessary
params_df = pd.concat({k: pd.DataFrame(v) for k, v in params.items()}, axis = 1)

# Ensures that all tickers are present in the final DataFrame, filling in with NaN where data are missing
params_df = params_df.reindex(index = excess_log_returns.index)

params_df

R_squared                                                    \
                ADBE       ADP      ABNB     GOOGL      GOOG      AMZN   
Date                                                                     
2013-12-31       NaN       NaN       NaN       NaN       NaN       NaN   
2014-01-02       NaN       NaN       NaN       NaN       NaN       NaN   
2014-01-03       NaN       NaN       NaN       NaN       NaN       NaN   
2014-01-06       NaN       NaN       NaN       NaN       NaN       NaN   
2014-01-07       NaN       NaN       NaN       NaN       NaN       NaN   
...              ...       ...       ...       ...       ...       ...   
2023-12-22  0.527815  0.133661  0.231776  0.430441  0.444822  0.465720   
2023-12-26  0.526925  0.133988  0.231999  0.431206  0.445641  0.464812   
2023-12-27  0.525128  0.135428  0.234837  0.429003  0.443564  0.462937   
2023-12-28  0.525989  0.136386  0.232808  0.428486  0.442613  0.460116   
2023-12-29  0.522396  0.134500  0.232771  0.422466  0.435976  0.451480   

                                                    ... variance_residual  \
                 AMD       AEP      AMGN       ADI  ...               FOX   
Date                                                ...                     
2013-12-31       NaN       NaN       NaN       NaN  ...               NaN   
2014-01-02       NaN       NaN       NaN       NaN  ...               NaN   
2014-01-03       NaN       NaN       NaN       NaN  ...               NaN   
2014-01-06       NaN       NaN       NaN       NaN  ...               NaN   
2014-01-07       NaN       NaN       NaN       NaN  ...               NaN   
...              ...       ...       ...       ...  ...               ...   
2023-12-22  0.474742  0.014408  0.053885  0.346127  ...          0.000184   
2023-12-26  0.479924  0.014546  0.053444  0.351466  ...          0.000183   
2023-12-27  0.478104  0.014584  0.053759  0.350505  ...          0.000182   
2023-12-28  0.474856  0.014489  0.052234  0.348207  ...          0.000182   
2023-12-29  0.486783  0.015208  0.050984  0.349175  ...          0.000182   

                                                                        \
               QRTEA      SRCL       WOR      STRM      ASAI      CHRS   
Date                                                                     
2013-12-31       NaN       NaN       NaN       NaN       NaN       NaN   
2014-01-02       NaN       NaN       NaN       NaN       NaN       NaN   
2014-01-03       NaN       NaN       NaN       NaN       NaN       NaN   
2014-01-06       NaN       NaN       NaN       NaN       NaN       NaN   
2014-01-07       NaN       NaN       NaN       NaN       NaN       NaN   
...              ...       ...       ...       ...       ...       ...   
2023-12-22  0.005148  0.000316  0.000437  0.009656  0.000925  0.004712   
2023-12-26  0.005134  0.000316  0.000436  0.009894  0.000924  0.004740   
2023-12-27  0.005045  0.000316  0.000435  0.009900  0.000898  0.005000   
2023-12-28  0.004978  0.000316  0.000436  0.009919  0.000898  0.005208   
2023-12-29  0.004976  0.000316  0.000436  0.009891  0.000885  0.005208   

                                          
                LOGI      CHKP     LBTYA  
Date                                      
2013-12-31       NaN       NaN       NaN  
2014-01-02       NaN       NaN       NaN  
2014-01-03       NaN       NaN       NaN  
2014-01-06       NaN       NaN       NaN  
2014-01-07       NaN       NaN       NaN  
...              ...       ...       ...  
2023-12-22  0.000369  0.000150  0.000380  
2023-12-26  0.000369  0.000150  0.000382  
2023-12-27  0.000370  0.000150  0.000382  
2023-12-28  0.000369  0.000148  0.000382  
2023-12-29  0.000369  0.000148  0.000382  

[2517 rows x 1624 columns]

In [ ]:
print(results["ABNB"].pvalues[:, 0].shape)
print(results["AAPL"].pvalues[:, 0].shape)
print(type(results["ABNB"].pvalues[:, 0]))

(767,)
(2516,)
<class 'numpy.ndarray'>


We also extract the p-values of the alpha parameter in order to check its significance later:

In [ ]:
params2 = {

    "alpha_significativity": {}
}

# Create a common index covering the entire date range
common_index = excess_log_returns.index

for ticker in tickers:
    if ticker in results:
        try:
            pvalues_alpha = pd.Series(results[ticker].pvalues[:, 0], index=results[ticker].model.data.row_labels)
            pvalues_alpha = pvalues_alpha.reindex(common_index)
            params2["alpha_significativity"][ticker] = pvalues_alpha
        except Exception as e:
            print(f"Error while processing the ticker {ticker}: {e}")
            params2["alpha_significativity"][ticker] = pd.Series(np.nan, index=common_index)

# Create a DataFrame for each key in params2
dfs = {k: pd.DataFrame(v) for k, v in params2.items()}

# Concatenate the DataFrames, filling with NaN where necessary
params_df2 = pd.concat(dfs, axis=1)

params_df2

alpha_significativity                                          \
                            ADBE       ADP      ABNB     GOOGL      GOOG   
Date                                                                       
2013-12-31                   NaN       NaN       NaN       NaN       NaN   
2014-01-02                   NaN       NaN       NaN       NaN       NaN   
2014-01-03                   NaN       NaN       NaN       NaN       NaN   
2014-01-06                   NaN       NaN       NaN       NaN       NaN   
2014-01-07                   NaN       NaN       NaN       NaN       NaN   
...                          ...       ...       ...       ...       ...   
2023-12-22              0.500867  0.679196  0.924755  0.986867  0.982693   
2023-12-26              0.493795  0.734114  0.792866  0.958928  0.963016   
2023-12-27              0.496969  0.674838  0.683108  0.989014  0.986099   
2023-12-28              0.533192  0.662820  0.710624  0.993712  0.976025   
2023-12-29              0.493910  0.688419  0.725361  0.981547  0.967280   

                                                              ...            \
                AMZN       AMD       AEP      AMGN       ADI  ...       FOX   
Date                                                          ...             
2013-12-31       NaN       NaN       NaN       NaN       NaN  ...       NaN   
2014-01-02       NaN       NaN       NaN       NaN       NaN  ...       NaN   
2014-01-03       NaN       NaN       NaN       NaN       NaN  ...       NaN   
2014-01-06       NaN       NaN       NaN       NaN       NaN  ...       NaN   
2014-01-07       NaN       NaN       NaN       NaN       NaN  ...       NaN   
...              ...       ...       ...       ...       ...  ...       ...   
2023-12-22  0.622348  0.911799  0.170235  0.928626  0.308594  ...  0.238298   
2023-12-26  0.658096  0.855354  0.185479  0.896605  0.259273  ...  0.207805   
2023-12-27  0.611837  0.906030  0.187104  0.877599  0.266430  ...  0.169243   
2023-12-28  0.569272  0.965528  0.199829  0.820602  0.287618  ...  0.195831   
2023-12-29  0.639531  0.944995  0.207863  0.828722  0.293099  ...  0.198821   

                                                                        \
               QRTEA      SRCL       WOR      STRM      ASAI      CHRS   
Date                                                                     
2013-12-31       NaN       NaN       NaN       NaN       NaN       NaN   
2014-01-02       NaN       NaN       NaN       NaN       NaN       NaN   
2014-01-03       NaN       NaN       NaN       NaN       NaN       NaN   
2014-01-06       NaN       NaN       NaN       NaN       NaN       NaN   
2014-01-07       NaN       NaN       NaN       NaN       NaN       NaN   
...              ...       ...       ...       ...       ...       ...   
2023-12-22  0.722069  0.879915  0.288424  0.297431  0.803673  0.084521   
2023-12-26  0.675171  0.922242  0.301135  0.369829  0.784676  0.093728   
2023-12-27  0.571571  0.995087  0.331275  0.355408  0.645383  0.156021   
2023-12-28  0.694472  0.985948  0.341780  0.332306  0.634484  0.233252   
2023-12-29  0.696692  0.980792  0.345989  0.293415  0.691748  0.237064   

                                          
                LOGI      CHKP     LBTYA  
Date                                      
2013-12-31       NaN       NaN       NaN  
2014-01-02       NaN       NaN       NaN  
2014-01-03       NaN       NaN       NaN  
2014-01-06       NaN       NaN       NaN  
2014-01-07       NaN       NaN       NaN  
...              ...       ...       ...  
2023-12-22  0.253117  0.839426  0.313527  
2023-12-26  0.251935  0.771081  0.354195  
2023-12-27  0.228287  0.803461  0.339745  
2023-12-28  0.249644  0.917496  0.386056  
2023-12-29  0.240161  0.921239  0.429479  

[2517 rows x 232 columns]

In [ ]:
df = pd.concat([params_df, params_df2], axis = 1)
df

R_squared                                                    \
                ADBE       ADP      ABNB     GOOGL      GOOG      AMZN   
Date                                                                     
2013-12-31       NaN       NaN       NaN       NaN       NaN       NaN   
2014-01-02       NaN       NaN       NaN       NaN       NaN       NaN   
2014-01-03       NaN       NaN       NaN       NaN       NaN       NaN   
2014-01-06       NaN       NaN       NaN       NaN       NaN       NaN   
2014-01-07       NaN       NaN       NaN       NaN       NaN       NaN   
...              ...       ...       ...       ...       ...       ...   
2023-12-22  0.527815  0.133661  0.231776  0.430441  0.444822  0.465720   
2023-12-26  0.526925  0.133988  0.231999  0.431206  0.445641  0.464812   
2023-12-27  0.525128  0.135428  0.234837  0.429003  0.443564  0.462937   
2023-12-28  0.525989  0.136386  0.232808  0.428486  0.442613  0.460116   
2023-12-29  0.522396  0.134500  0.232771  0.422466  0.435976  0.451480   

                                                    ... alpha_significativity  \
                 AMD       AEP      AMGN       ADI  ...                   FOX   
Date                                                ...                         
2013-12-31       NaN       NaN       NaN       NaN  ...                   NaN   
2014-01-02       NaN       NaN       NaN       NaN  ...                   NaN   
2014-01-03       NaN       NaN       NaN       NaN  ...                   NaN   
2014-01-06       NaN       NaN       NaN       NaN  ...                   NaN   
2014-01-07       NaN       NaN       NaN       NaN  ...                   NaN   
...              ...       ...       ...       ...  ...                   ...   
2023-12-22  0.474742  0.014408  0.053885  0.346127  ...              0.238298   
2023-12-26  0.479924  0.014546  0.053444  0.351466  ...              0.207805   
2023-12-27  0.478104  0.014584  0.053759  0.350505  ...              0.169243   
2023-12-28  0.474856  0.014489  0.052234  0.348207  ...              0.195831   
2023-12-29  0.486783  0.015208  0.050984  0.349175  ...              0.198821   

                                                                        \
               QRTEA      SRCL       WOR      STRM      ASAI      CHRS   
Date                                                                     
2013-12-31       NaN       NaN       NaN       NaN       NaN       NaN   
2014-01-02       NaN       NaN       NaN       NaN       NaN       NaN   
2014-01-03       NaN       NaN       NaN       NaN       NaN       NaN   
2014-01-06       NaN       NaN       NaN       NaN       NaN       NaN   
2014-01-07       NaN       NaN       NaN       NaN       NaN       NaN   
...              ...       ...       ...       ...       ...       ...   
2023-12-22  0.722069  0.879915  0.288424  0.297431  0.803673  0.084521   
2023-12-26  0.675171  0.922242  0.301135  0.369829  0.784676  0.093728   
2023-12-27  0.571571  0.995087  0.331275  0.355408  0.645383  0.156021   
2023-12-28  0.694472  0.985948  0.341780  0.332306  0.634484  0.233252   
2023-12-29  0.696692  0.980792  0.345989  0.293415  0.691748  0.237064   

                                          
                LOGI      CHKP     LBTYA  
Date                                      
2013-12-31       NaN       NaN       NaN  
2014-01-02       NaN       NaN       NaN  
2014-01-03       NaN       NaN       NaN  
2014-01-06       NaN       NaN       NaN  
2014-01-07       NaN       NaN       NaN  
...              ...       ...       ...  
2023-12-22  0.253117  0.839426  0.313527  
2023-12-26  0.251935  0.771081  0.354195  
2023-12-27  0.228287  0.803461  0.339745  
2023-12-28  0.249644  0.917496  0.386056  
2023-12-29  0.240161  0.921239  0.429479  

[2517 rows x 1856 columns]

In [ ]:
df.to_csv("Data/parameters.csv", index = True)